In [3]:
import pandas as pd
import numpy as np

from pathlib import Path
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

In [4]:
from utils import load_car_data

df = load_car_data(Path('../data/fuel_consumption.csv'))

features = ['make', 'release_year', 'vehicle_class', 'fuel_type', 'transmission_type', 'gears', 'engine_size', 'cylinders']
numerical_features = ['release_year', 'gears', 'engine_size', 'cylinders']
categorical_features = ['make', 'vehicle_class', 'fuel_type', 'transmission_type']

targets = ['emissions', 'fc_mixed', 'fc_city', 'fc_highway']

X = df[features]
Y = df[targets]

2025-01-24 02:57:49.362 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-01-24 02:57:49.374 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-24 02:57:49.421 
  command:

    streamlit run /Users/mathisderenne/GitHub/Fuel-Consumption-App/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-24 02:57:49.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-24 02:57:49.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-24 02:57:49.501 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-24 02:57:49.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


### Model definition

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso

preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('numerical', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_features)
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso(max_iter=10000))
])

param = {
    'regressor__alpha': np.logspace(-4, 4, 5)
}

In [6]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipeline, param, cv=5, n_jobs=-1, refit = True)
grid.fit(X, Y)

print(f"Best parameters : {grid.best_params_}")
print(f"Best score : {grid.best_score_:0.2f}")

/Users/mathisderenne/GitHub/Fuel-Consumption-App/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 107980.81709404662, tolerance: 6285.037067451782
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/mathisderenne/GitHub/Fuel-Consumption-App/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 998960.1501836069, tolerance: 6209.437799002492
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/mathisderenne/GitHub/Fuel-Consumption-App/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1111006.7056265408, tolerance: 6387.928191166539
  model = cd_

Best parameters : {'regressor__alpha': 0.0001}
Best score : 0.77


### Save model

In [7]:
import joblib

joblib.dump(grid.best_estimator_, '../data/lasso_regression.pkl')

['../data/lasso_regression.pkl']